In [1]:
import pandas as pd
import numpy as np

import ast 

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data_credits=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [3]:
data_credits

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [4]:
data_movies=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [5]:
data_movies.shape

(4803, 20)

In [6]:
movies=data_movies.merge(data_credits,on='title')

In [7]:
movies.sample(4)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
2772,0,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 35, ""...",NaN,9683,"[{""id"": 725, ""name"": ""lovesickness""}, {""id"": 8...",en,Bubble Boy,Jimmy is young man who was born without an imm...,10.131123,"[{""name"": ""Touchstone Pictures"", ""id"": 9195}]",...,84.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Life is an adventure. Don't blow it.,Bubble Boy,5.1,163,9683,"[{""cast_id"": 1, ""character"": ""Jimmy Livingston...","[{""credit_id"": ""52fe451bc3a36847f80bcc9f"", ""de..."
4643,250000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 35, ""name...",NaN,32222,"[{""id"": 941, ""name"": ""tradition""}, {""id"": 958,...",en,American Desi,This is a fun-loving romantic comedy reminisce...,0.126304,[],...,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,American Desi,5.0,4,32222,"[{""cast_id"": 1, ""character"": ""Ajay Pandya"", ""c...","[{""credit_id"": ""57fd2ab3c3a3681b91000b83"", ""de..."
1124,45000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 35, ""...",NaN,7443,"[{""id"": 2479, ""name"": ""chicken""}, {""id"": 6089,...",en,Chicken Run,Having been hopelessly repressed and facing ev...,31.789029,"[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...",...,84.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,This ain't no chick flick. It's poultry in mot...,Chicken Run,6.5,1150,7443,"[{""cast_id"": 3, ""character"": ""Rocky (Voice)"", ...","[{""credit_id"": ""52fe447cc3a36847f80990f1"", ""de..."
4621,300000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name...",NaN,2786,"[{""id"": 90, ""name"": ""paris""}, {""id"": 5918, ""na...",fr,Pierrot le fou,Pierrot escapes his boring society and travels...,7.791898,"[{""name"": ""Dino de Laurentiis Cinematografica""...",...,110.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}]",Released,NaN,Pierrot le Fou,7.6,127,2786,"[{""cast_id"": 12, ""character"": ""Ferdinand-Pierr...","[{""credit_id"": ""52fe436dc3a36847f805355b"", ""de..."


In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

## Feature Engineering and Data Preprocessing

In [9]:
#finding the required column features
# genres, keywords, overview, title, movie_id,cast,crew

movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [10]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [11]:
movies.dropna(inplace=True)

In [12]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [13]:
# checking duplicate data
movies.duplicated().sum()

0

In [14]:
movies['genres'].iloc[1]

'[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]'

We want genres in the format like [Adventure, Fantasy, Action]

In [15]:
def convert(item):
    L=[]
    for i in ast.literal_eval(item):
        L.append(i['name'])
    return L

# we want item in a list but it is in string 
# so we use ast.literal_eval to convert it into list

In [16]:
# Checking the convert function
convert('[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]')

['Adventure', 'Fantasy', 'Action']

In [17]:
movies['genres'].apply(convert)

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4806, dtype: object

In [18]:
movies['genres']=movies['genres'].apply(convert)

In [19]:
movies['keywords'][1]

'[{"id": 270, "name": "ocean"}, {"id": 726, "name": "drug abuse"}, {"id": 911, "name": "exotic island"}, {"id": 1319, "name": "east india trading company"}, {"id": 2038, "name": "love of one\'s life"}, {"id": 2052, "name": "traitor"}, {"id": 2580, "name": "shipwreck"}, {"id": 2660, "name": "strong woman"}, {"id": 3799, "name": "ship"}, {"id": 5740, "name": "alliance"}, {"id": 5941, "name": "calypso"}, {"id": 6155, "name": "afterlife"}, {"id": 6211, "name": "fighter"}, {"id": 12988, "name": "pirate"}, {"id": 157186, "name": "swashbuckler"}, {"id": 179430, "name": "aftercreditsstinger"}]'

We only want name of all the keywords

In [20]:
movies['keywords'].apply(convert)

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united states–mexico barrier, legs, arms, pap...
4805                                                   []
4806    [date, love at first sight, narration, investi...
4807                                                   []
4808            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4806, dtype: object

In [21]:
movies['keywords']=movies['keywords'].apply(convert)

In [22]:
movies['cast'][1]

'[{"cast_id": 4, "character": "Captain Jack Sparrow", "credit_id": "52fe4232c3a36847f800b50d", "gender": 2, "id": 85, "name": "Johnny Depp", "order": 0}, {"cast_id": 5, "character": "Will Turner", "credit_id": "52fe4232c3a36847f800b511", "gender": 2, "id": 114, "name": "Orlando Bloom", "order": 1}, {"cast_id": 6, "character": "Elizabeth Swann", "credit_id": "52fe4232c3a36847f800b515", "gender": 1, "id": 116, "name": "Keira Knightley", "order": 2}, {"cast_id": 12, "character": "William \\"Bootstrap Bill\\" Turner", "credit_id": "52fe4232c3a36847f800b52d", "gender": 2, "id": 1640, "name": "Stellan Skarsg\\u00e5rd", "order": 3}, {"cast_id": 10, "character": "Captain Sao Feng", "credit_id": "52fe4232c3a36847f800b525", "gender": 2, "id": 1619, "name": "Chow Yun-fat", "order": 4}, {"cast_id": 9, "character": "Captain Davy Jones", "credit_id": "52fe4232c3a36847f800b521", "gender": 2, "id": 2440, "name": "Bill Nighy", "order": 5}, {"cast_id": 7, "character": "Captain Hector Barbossa", "credit_

We have to take first three main cast name

In [23]:
# for finding top 3 actors name from cast column
def cast_name(obj):
    count=0
    L=[]
    for i in ast.literal_eval(obj):
        if count!=3:
            L.append(i['name'])
            count+=1
        else:
            break
    return L

In [24]:
movies['cast']=movies['cast'].apply(cast_name)

In [25]:
movies['crew'][1]

'[{"credit_id": "52fe4232c3a36847f800b579", "department": "Camera", "gender": 2, "id": 120, "job": "Director of Photography", "name": "Dariusz Wolski"}, {"credit_id": "52fe4232c3a36847f800b4fd", "department": "Directing", "gender": 2, "id": 1704, "job": "Director", "name": "Gore Verbinski"}, {"credit_id": "52fe4232c3a36847f800b54f", "department": "Production", "gender": 2, "id": 770, "job": "Producer", "name": "Jerry Bruckheimer"}, {"credit_id": "52fe4232c3a36847f800b503", "department": "Writing", "gender": 2, "id": 1705, "job": "Screenplay", "name": "Ted Elliott"}, {"credit_id": "52fe4232c3a36847f800b509", "department": "Writing", "gender": 2, "id": 1706, "job": "Screenplay", "name": "Terry Rossio"}, {"credit_id": "52fe4232c3a36847f800b57f", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "52fe4232c3a36847f800b585", "department": "Editing", "gender": 2, "id": 1722, "job": "Editor", "name": "Craig Wood"}, {"credit_id": "52f

We want only director from crew column

In [26]:
def crew_dir(item):
    L=[]
    for i in ast.literal_eval(item):
        if i['job']=='Director':
            L.append(i['name'])
    return L

In [27]:
crew_dir(movies['crew'][1])

['Gore Verbinski']

In [28]:
movies['crew']=movies['crew'].apply(crew_dir)

In [29]:
movies.sample(3)

,movie_id,title,overview,genres,keywords,cast,crew
1251,15556,At First Sight,A blind man has an operation to regain his sig...,"[Drama, Romance]","[blind, massage therapist]","[Val Kilmer, Mira Sorvino, Kelly McGillis]",[Irwin Winkler]
4584,1282,Dogtown and Z-Boys,Dogtown and Z-Boys follows the evolution of sk...,[Documentary],"[skateboarding, independent film]","[Sean Penn, Jay Adams, Henry Rollins]",[Stacy Peralta]
1620,12763,Take the Lead,A former professional dancer volunteers to tea...,[Music],"[dancing master, dance, musical, woman director]","[Antonio Banderas, Rob Brown, Yaya DaCosta]",[Liz Friedlander]


In [30]:
# converting the overviev to list for adding all the column together
# as all other columns is also in list then it is easy to add then
movies['overview']=movies['overview'].apply(lambda x: x.split())

In [31]:
# removing the white spaces from the all the columns for avoiding the confusion
movies['genres']=movies['genres'].apply(lambda x:[i.replace(' ','') for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(' ','') for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(' ','') for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(' ','') for i in x])

In [32]:
movies.sample(2)

,movie_id,title,overview,genres,keywords,cast,crew
2814,9552,The Exorcist,"[12-year-old, Regan, MacNeil, begins, to, adap...","[Drama, Horror, Thriller]","[exorcism, holywater, religionandsupernatural,...","[LindaBlair, MaxvonSydow, EllenBurstyn]",[WilliamFriedkin]
3756,8337,They Live,"[Nada,, a, down-on-his-luck, construction, wor...","[Action, Horror, ScienceFiction, Thriller]","[dystopia, socialcommentary, alieninvasion, su...","[RoddyPiper, KeithDavid, MegFoster]",[JohnCarpenter]


In [33]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [34]:
movies.sample(2)

,movie_id,title,overview,genres,keywords,cast,crew,tags
4532,36825,The R.M.,"[Jared, Phelps, (Kirby, Heyborne), has, comple...","[Comedy, Crime, Family]",[],[KirbyHeyborne],[KurtHale],"[Jared, Phelps, (Kirby, Heyborne), has, comple..."
323,37786,Sex and the City 2,"[Carrie,, Charlotte,, and, Miranda, are, all, ...","[Comedy, Drama, Romance]",[],"[SarahJessicaParker, KristinDavis, CynthiaNixon]",[MichaelPatrickKing],"[Carrie,, Charlotte,, and, Miranda, are, all, ..."


In [35]:
df=movies[['movie_id','title','tags']]

In [36]:
df

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [37]:
# converting list to the string
df['tags']=df['tags'].apply(lambda x:' '.join(x))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
# converting all into lower case
df['tags']=df['tags'].apply(lambda x: x.lower())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
df

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [40]:
# Creating the countvectorizer object
cv=CountVectorizer(max_features=5000,stop_words='english')

In [41]:
# text vectorization
vectors=cv.fit_transform(df['tags']).toarray()

In [42]:
# to get most recurrent 5000 words
cv.get_feature_names()[50:100]

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['abducted',
 'abigailbreslin',
 'abilities',
 'ability',
 'able',
 'aboard',
 'abuse',
 'abusive',
 'academic',
 'academy',
 'accept',
 'accepted',
 'accepts',
 'access',
 'accident',
 'accidental',
 'accidentally',
 'accompanied',
 'accomplish',
 'account',
 'accountant',
 'accused',
 'ace',
 'achieve',
 'act',
 'acting',
 'action',
 'actionhero',
 'actions',
 'activist',
 'activities',
 'activity',
 'actor',
 'actors',
 'actress',
 'acts',
 'actual',
 'actually',
 'adam',
 'adams',
 'adamsandler',
 'adamshankman',
 'adaptation',
 'adapted',
 'addict',
 'addicted',
 'addiction',
 'adolescence',
 'adopt',
 'adopted']

As you see there are lots off same words having same meaning are occured
like "actual","actually"
And we have to remove such words
So we have to do steamming

## Steamming

In [43]:
# object of porterstemmer for steamming
ps=PorterStemmer()

In [44]:
def stemming(text):
    L=[]
    for i in text.split():
        L.append(ps.stem(i))
    return " ".join(L)

In [45]:
df['tags']=df['tags'].apply(stemming)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [47]:
vectors=cv.fit_transform(df['tags']).toarray()

In [48]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [49]:
cv.get_feature_names()[50:100]

['500',
 '60',
 '70',
 '80',
 'aaron',
 'aaroneckhart',
 'abandon',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'abov',
 'abus',
 'academ',
 'academi',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'account',
 'accus',
 'ace',
 'achiev',
 'acquaint',
 'act',
 'action',
 'actionhero',
 'activ',
 'activist',
 'activities',
 'actor',
 'actress',
 'actual',
 'ad',
 'adam',
 'adamsandl',
 'adamshankman',
 'adapt',
 'add',
 'addict',
 'adjust',
 'admir',
 'admit',
 'adolesc',
 'adopt',
 'ador']

## Cosine Similarity

In [50]:
# cosine similarity to check the similarity between different movies
similarity=cosine_similarity(vectors)

In [51]:
similarity

array([[1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
        0.        ],
       [0.08346223, 1.        , 0.06063391, ..., 0.02378257, 0.        ,
        0.02615329],
       [0.0860309 , 0.06063391, 1.        , ..., 0.02451452, 0.        ,
        0.        ],
       ...,
       [0.04499213, 0.02378257, 0.02451452, ..., 1.        , 0.03962144,
        0.04229549],
       [0.        , 0.        , 0.        , ..., 0.03962144, 1.        ,
        0.08714204],
       [0.        , 0.02615329, 0.        , ..., 0.04229549, 0.08714204,
        1.        ]])

## Model Building

In [52]:
def recommand(movie):
    movie_index=df[df['title']==movie].index[0]
    distance=similarity[movie_index]
    movies_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(df.iloc[i[0]]['title'])

In [53]:
recommand("Spectre")

Quantum of Solace
Skyfall
Never Say Never Again
From Russia with Love
Octopussy
